In [1]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

import re
import os
import pickle

In [2]:
from google.auth.exceptions import RefreshError

In [5]:
"""
CountryCode
language

1. How to program inPython.
2. Python for beginers
3. Numpy tutorials
5. Python strings
6. Advanced python programming
7. How to solve x

"""

'\nCountryCode\nlanguage\n\n1. How to program inPython.\n2. Python for beginers\n3. Numpy tutorials\n5. Python strings\n6. Advanced python programming\n7. How to solve x\n\n'

In [6]:
"""
youtube = YouTube()
youtube.authenticate(credentials_file='', token_path='')
youtube = YouTube(credentials_file='', token_path='')
youtube.search_video(): query -> List[YouTubeVideo]
youtube.search_channel(): query -> List[YouTubeChannel]
youtube.search_playlist(): query -> List[PlayList]
youtube.YouTubeVideo(video_id='') -> YouTubeVideo
youtube.YouTubeVideo(video_url='')
youtube.errors.
youtube.utils
"""

"\nyoutube = YouTube()\nyoutube.authenticate(credentials_file='', token_path='')\nyoutube = YouTube(credentials_file='', token_path='')\nyoutube.search_video(): query -> List[YouTubeVideo]\nyoutube.search_channel(): query -> List[YouTubeChannel]\nyoutube.search_playlist(): query -> List[PlayList]\nyoutube.YouTubeVideo(video_id='') -> YouTubeVideo\nyoutube.YouTubeVideo(video_url='')\nyoutube.errors.\nyoutube.utils\n"

In [3]:
#api.constants.YouTubeAPIContants
class YouTubeAPIConstants:
    TOKEN_FILE = 'token.pickle'
    API_SERVICE_NAME = 'youtube'
    API_VERSION = 'v3'
    SCOPES = ["https://www.googleapis.com/auth/youtube.force-ssl"]

class Authenticate:
    """Handle the YouTube authentication process."""
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
    __TOKEN_FILE = YouTubeAPIConstants.TOKEN_FILE
    __API_SERVICE_NAME = YouTubeAPIConstants.API_SERVICE_NAME
    __API_VERSION = YouTubeAPIConstants.API_VERSION
    __SCOPES = YouTubeAPIConstants.SCOPES
    
    def __init__(self, client_secrets_file: str , api_token_path: str = ''):
        """Create the auth object."""
        self.__credentials = None
        
        self.__verify_client_secret_file(client_secrets_file)
        self.__client_secrets_file = client_secrets_file
        
        if not api_token_path or not os.path.exists(api_token_path):
            self.__api_token_path = self.__get_default_api_token_path()
        else:
            self.__api_token_path = api_token_path
            
    def __verify_client_secret_file(self, client_secrets_file: str) -> None:
        """Verfy the client secret file."""
        if not client_secrets_file:
            raise ValueError('The clients secret file path has to be provided.')
        if not isinstance(client_secrets_file, str):
            raise TypeError('The clients secret file should be a string.')
        if not os.path.exists(client_secrets_file):
            raise ValueError(f'The path {client_secrets_file} does not exist!')
        
    def __get_default_api_token_path(self):
        """Generate the default api token file location."""
        current_user_home_dir = os.path.expanduser('~')
        api_token_path = os.path.join(current_user_home_dir, self.__TOKEN_FILE)
        return api_token_path
    
    def __authenticate_youtube(self):
        """Authenticate the YouTube API."""
        if os.path.exists(self.__api_token_path):
            with open(self.__api_token_path, "rb") as token:
                self.__credentials = pickle.load(token)
        # if there are no (valid) credentials availablle, let the user log in.
        if not self.__credentials or not self.__credentials.valid:
            if self.__credentials and self.__credentials.expired and self.__credentials.refresh_token:
                self.__credentials.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(self.__client_secrets_file, self.__SCOPES)
                self.__credentials = flow.run_local_server(port=0)
            # save the credentials for the next run
            with open(self.__api_token_path, "wb") as token:
                pickle.dump(self.__credentials, token)

        return build(self.__API_SERVICE_NAME, self.__API_VERSION, credentials=self.__credentials)
    
    def authenticate(self):
        try:
            youtube_client =  self.__authenticate_youtube()
        #TODO generate a more specific Exception
        except RefreshError as e:
            raise Exception('The token is expired. Kindly generate a new one.')
        else:
            return youtube_client
        

In [4]:
class YouTubeSearchQuery:
    """A query to pass to the search resource."""
    
    def __init__(self, query_string: str):
        self.__query_string = query_string
        
    @property
    def query_string(self):
        return self.__query_string
    
    @query_string.setter
    def query_string(self, query_str: str):
        if not query_str:
            raise ValueError('The query string has to be provided')
        if not isinstance(query_str, str):
            raise TypeError('The query string has to be a string')
        self.__query_string = query_str

        
class YouTubeVideoSearchQuery(YouTubeSearchQuery):
    pass
        
"""
Returns a collection of search results that match the query parameters specified in 
the API request. By default, a search result set identifies matching video, channel, 
and playlist resources, but you can also configure queries to only retrieve a specific 
type of resource.

{
  "kind": "youtube#searchResult",
  "etag": etag,
  "id": {
    "kind": string,
    "videoId": string,
    "channelId": string,
    "playlistId": string
  },
  "snippet": {
    "publishedAt": datetime,
    "channelId": string,
    "title": string,
    "description": string,
    "thumbnails": {
      (key): {
        "url": string,
        "width": unsigned integer,
        "height": unsigned integer
      }
    },
    "channelTitle": string,
    "liveBroadcastContent": string
  }
}

"""

'\nReturns a collection of search results that match the query parameters specified in \nthe API request. By default, a search result set identifies matching video, channel, \nand playlist resources, but you can also configure queries to only retrieve a specific \ntype of resource.\n\n{\n  "kind": "youtube#searchResult",\n  "etag": etag,\n  "id": {\n    "kind": string,\n    "videoId": string,\n    "channelId": string,\n    "playlistId": string\n  },\n  "snippet": {\n    "publishedAt": datetime,\n    "channelId": string,\n    "title": string,\n    "description": string,\n    "thumbnails": {\n      (key): {\n        "url": string,\n        "width": unsigned integer,\n        "height": unsigned integer\n      }\n    },\n    "channelTitle": string,\n    "liveBroadcastContent": string\n  }\n}\n\n'

In [5]:
class YouTubeSearchType:
    VIDEO = 'video'
    CHANNEL = 'channel'
    PLAYLIST = 'playlist'
    
class YouTubeSearch:
    MAX_RESULTS = 10
    REGION_CODE = 'US'
    
    def __init__(self, *args, **kwargs):
        pass

class VideoSearch(YouTubeSearch):
    def __init__(self, query_string: str):
        self.__type = YouTubeSearchType.VIDEO
        self.__query = YouTubeSearchQuery(query_string)
        
    def __get_query(self):
        return self.__query.query_string
        
    def basic_info(self):
        basic_info_params = self.__generate_basic_info_params()
        return basic_info_params
    
    def advanced_info(self):
        pass
    
    def all_info(self):
        pass
    
    def __generate_basic_info_params(self, next_page_token=''):
        basic_info_params = dict(
            part='id,snippet',
            type=self.__type,
            q=self.__get_query(),
            maxResults=self.MAX_RESULTS,
            regionCode=self.REGION_CODE
        ) 
        if next_page_token:
            basic_info_params['pageToken'] = next_page_token
        return basic_info_params
    
    def search_video(self, youtube_client, search_type='basic', next_page_token=''):
        search_response = None
        if search_type == 'basic':
            basic_info_params = self.__generate_basic_info_params(next_page_token='')
            search_request = youtube_client.search().list(
                **basic_info_params
            )
            search_response = search_request.execute()
        return search_response
    
    @staticmethod
    def parse_basic_response(search_result: dict) -> list[dict[str, str]]:
        video_details = []
        video_results = search_result['items']
        for video_result in video_results:
            video = {}
            video_id = video_result['id']
            video_snippet = video_result['snippet']
            video['id'] = video_id['videoId']
            video['title'] = video_snippet['title']
            video['description'] = video_snippet['description']
            video['channelTitle'] = video_snippet['channelTitle']
            video_details.append(video)
        return video_details
        

In [16]:
class PlaylistSearch(YouTubeSearch):
    def __init__(self, query_string: str):
        self.__type = YouTubeSearchType.PLAYLIST
        self.__query = YouTubeSearchQuery(query_string)
        
    def __get_query(self):
        return self.__query.query_string
        
    def basic_info(self):
        basic_info_params = self.__generate_basic_info_params()
        return basic_info_params
    
    def advanced_info(self):
        pass
    
    def all_info(self):
        pass
    
    def __generate_basic_info_params(self, next_page_token=''):
        basic_info_params = dict(
            part='id,snippet',
            type=self.__type,
            q=self.__get_query(),
            maxResults=self.MAX_RESULTS,
            regionCode=self.REGION_CODE
        ) 
        if next_page_token:
            basic_info_params['pageToken'] = next_page_token
        return basic_info_params
    
    def search_playlist(self, youtube_client, search_type='basic', next_page_token=''):
        search_response = None
        if search_type == 'basic':
            basic_info_params = self.__generate_basic_info_params(next_page_token='')
            search_request = youtube_client.search().list(
                **basic_info_params
            )
            search_response = search_request.execute()
        return search_response
    
    @staticmethod
    def parse_basic_response(search_result: dict) -> list[dict[str, str]]:
        video_details = []
        video_results = search_result['items']
        for video_result in video_results:
            video = {}
            video_id = video_result['id']
            video_snippet = video_result['snippet']
            video['id'] = video_id['videoId']
            video['title'] = video_snippet['title']
            video['description'] = video_snippet['description']
            video['channelTitle'] = video_snippet['channelTitle']
            video_details.append(video)
        return video_details
        

In [23]:
class ChannelSearch(YouTubeSearch):
    def __init__(self, query_string: str):
        self.__type = YouTubeSearchType.CHANNEL
        self.__query = YouTubeSearchQuery(query_string)
        
    def __get_query(self):
        return self.__query.query_string
        
    def basic_info(self):
        basic_info_params = self.__generate_basic_info_params()
        return basic_info_params
    
    def advanced_info(self):
        pass
    
    def all_info(self):
        pass
    
    def __generate_basic_info_params(self, next_page_token=''):
        basic_info_params = dict(
            part='id,snippet',
            type=self.__type,
            q=self.__get_query(),
            maxResults=self.MAX_RESULTS,
            regionCode=self.REGION_CODE
        ) 
        if next_page_token:
            basic_info_params['pageToken'] = next_page_token
        return basic_info_params
    
    def search_channel(self, youtube_client, search_type='basic', next_page_token=''):
        search_response = None
        if search_type == 'basic':
            basic_info_params = self.__generate_basic_info_params(next_page_token='')
            search_request = youtube_client.search().list(
                **basic_info_params
            )
            search_response = search_request.execute()
        return search_response
    
    @staticmethod
    def parse_basic_response(search_result: dict) -> list[dict[str, str]]:
        video_details = []
        video_results = search_result['items']
        for video_result in video_results:
            video = {}
            video_id = video_result['id']
            video_snippet = video_result['snippet']
            video['id'] = video_id['videoId']
            video['title'] = video_snippet['title']
            video['description'] = video_snippet['description']
            video['channelTitle'] = video_snippet['channelTitle']
            video_details.append(video)
        return video_details
        

In [6]:
class YouTube:
    def __init__(self, credentials_file='', token_path=''):
        self.__auth = Authenticate(credentials_file, token_path)
        self.__youtube_client = None
        self.__is_authenticated = False
        
    def is_authenticated(self):
        return self.__is_authenticated
        
    def authenticate(self, credentials_file='', token_path=''):
        self.__youtube_client = self.__auth.authenticate()
        self.__is_authenticated = True
        
    def get_youtube(self):
        return self.__youtube_client
        
    def search_video(self, query_string: str) -> list[str]:
        video_search = VideoSearch('python')
        search_results = video_search.search_video(self.__youtube_client)
        parsed_results = video_search.parse_basic_response(search_results)
        return parsed_results
        
    def find_video(self, video: str, by: str = 'url'):
        """Find a video by url or id."""
        if by:
            if not isinstance(by, str):
                raise ValueError('by must be a string')
            if by not in ['url', 'id']:
                raise ValueError('by must be either id or url')
        if by == 'url':
            return self.__get_video_id(video)
    
    @staticmethod
    def __get_video_id(video_url: str) -> str:
        """Get vdeo ID from video url"""
        if not video_url:
            raise ValueError('The video_ur has to be provided.')
        if not isinstance(video_url, str):
            raise TypeError('Te video_url has to be a string.')
        if '=' not in video_url:
            url_format = 'https://www.youtube.com/watch?v=Dqdu-FsBk0s'
            raise ValueError('Te video_url should be of the format "{url_format}"')
        video_url = video_url.split('=')[1]
        return video_url

In [7]:
youtube = YouTube(credentials_file='/home/lyle/Downloads/client_secret.json', 
                  token_path='')
youtube.authenticate()
youtube_client = youtube.get_youtube()

In [8]:
youtube.search_video('python')

[{'id': 'rfscVS0vtbw',
  'title': 'Learn Python - Full Course for Beginners [Tutorial]',
  'description': "This course will give you a full introduction into all of the core concepts in python. Follow along with the videos and you'll be a ...",
  'channelTitle': 'freeCodeCamp.org'},
 {'id': 'ChoeKcjlYuI',
  'title': 'Learn Python with ChatGPT',
  'description': 'Learn Python really FAST with ChatGPT . . For more content like this, subscribe to our channel: ...',
  'channelTitle': 'Power Couple'},
 {'id': '_uQrJ0TkZlc',
  'title': 'Python Tutorial - Python Full Course for Beginners',
  'description': 'Python tutorial - Python full course for beginners - Go from Zero to Hero with Python (includes machine learning & web ...',
  'channelTitle': 'Programming with Mosh'},
 {'id': 'kqtD5dpn9C8',
  'title': 'Python for Beginners - Learn Python in 1 Hour',
  'description': 'This Python tutorial for beginners show how to get started with Python quickly. Learn to code in 1 hour! Watch this tutori

In [137]:
video_search = VideoSearch('python')

In [138]:
video_search.basic_info()

{'part': 'id,snippet',
 'type': 'video',
 'q': 'python',
 'maxResults': 10,
 'regionCode': 'US'}

In [139]:
search_results = video_search.search_video(youtube_client)

In [140]:
search_results

{'kind': 'youtube#searchListResponse',
 'etag': 'VHpffh0mqYUSh6zneWB1zfIuyz0',
 'nextPageToken': 'CAoQAA',
 'regionCode': 'US',
 'pageInfo': {'totalResults': 1000000, 'resultsPerPage': 10},
 'items': [{'kind': 'youtube#searchResult',
   'etag': '6wSaSCZ6hvFW4Ug3JMCbOuDqoxo',
   'id': {'kind': 'youtube#video', 'videoId': 'rfscVS0vtbw'},
   'snippet': {'publishedAt': '2018-07-11T18:00:42Z',
    'channelId': 'UC8butISFwT-Wl7EV0hUK0BQ',
    'title': 'Learn Python - Full Course for Beginners [Tutorial]',
    'description': "This course will give you a full introduction into all of the core concepts in python. Follow along with the videos and you'll be a ...",
    'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/rfscVS0vtbw/default.jpg',
      'width': 120,
      'height': 90},
     'medium': {'url': 'https://i.ytimg.com/vi/rfscVS0vtbw/mqdefault.jpg',
      'width': 320,
      'height': 180},
     'high': {'url': 'https://i.ytimg.com/vi/rfscVS0vtbw/hqdefault.jpg',
      'width': 480

In [141]:
video_search.parse_basic_response(search_results)

[{'id': 'rfscVS0vtbw',
  'title': 'Learn Python - Full Course for Beginners [Tutorial]',
  'description': "This course will give you a full introduction into all of the core concepts in python. Follow along with the videos and you'll be a ...",
  'channelTitle': 'freeCodeCamp.org'},
 {'id': 'kqtD5dpn9C8',
  'title': 'Python for Beginners - Learn Python in 1 Hour',
  'description': 'This Python tutorial for beginners show how to get started with Python quickly. Learn to code in 1 hour! Watch this tutorial get ...',
  'channelTitle': 'Programming with Mosh'},
 {'id': '_uQrJ0TkZlc',
  'title': 'Python Tutorial - Python Full Course for Beginners',
  'description': 'Python tutorial - Python full course for beginners - Go from Zero to Hero with Python (includes machine learning & web ...',
  'channelTitle': 'Programming with Mosh'},
 {'id': 'b093aqAZiPU',
  'title': '👩\u200d💻 Python for Beginners Tutorial',
  'description': 'In this step-by-step Python for beginners tutorial, learn how you c

In [144]:
len(video_search.parse_basic_response(search_results))

10

In [145]:
search_results = video_search.search_video(youtube_client)
next_page_token = search_results['nextPageToken']
video_search.parse_basic_response(search_results)

[{'id': 'rfscVS0vtbw',
  'title': 'Learn Python - Full Course for Beginners [Tutorial]',
  'description': "This course will give you a full introduction into all of the core concepts in python. Follow along with the videos and you'll be a ...",
  'channelTitle': 'freeCodeCamp.org'},
 {'id': 'kqtD5dpn9C8',
  'title': 'Python for Beginners - Learn Python in 1 Hour',
  'description': 'This Python tutorial for beginners show how to get started with Python quickly. Learn to code in 1 hour! Watch this tutorial get ...',
  'channelTitle': 'Programming with Mosh'},
 {'id': '_uQrJ0TkZlc',
  'title': 'Python Tutorial - Python Full Course for Beginners',
  'description': 'Python tutorial - Python full course for beginners - Go from Zero to Hero with Python (includes machine learning & web ...',
  'channelTitle': 'Programming with Mosh'},
 {'id': 'b093aqAZiPU',
  'title': '👩\u200d💻 Python for Beginners Tutorial',
  'description': 'In this step-by-step Python for beginners tutorial, learn how you c

In [146]:
search_results = video_search.search_video(youtube_client, next_page_token=next_page_token)
next_page_token = search_results['nextPageToken']
video_search.parse_basic_response(search_results)

[{'id': 'rfscVS0vtbw',
  'title': 'Learn Python - Full Course for Beginners [Tutorial]',
  'description': "This course will give you a full introduction into all of the core concepts in python. Follow along with the videos and you'll be a ...",
  'channelTitle': 'freeCodeCamp.org'},
 {'id': 'kqtD5dpn9C8',
  'title': 'Python for Beginners - Learn Python in 1 Hour',
  'description': 'This Python tutorial for beginners show how to get started with Python quickly. Learn to code in 1 hour! Watch this tutorial get ...',
  'channelTitle': 'Programming with Mosh'},
 {'id': '_uQrJ0TkZlc',
  'title': 'Python Tutorial - Python Full Course for Beginners',
  'description': 'Python tutorial - Python full course for beginners - Go from Zero to Hero with Python (includes machine learning & web ...',
  'channelTitle': 'Programming with Mosh'},
 {'id': 'b093aqAZiPU',
  'title': '👩\u200d💻 Python for Beginners Tutorial',
  'description': 'In this step-by-step Python for beginners tutorial, learn how you c

In [17]:
playlist_search = PlaylistSearch('python')
search_results = playlist_search.search_playlist(youtube_client)

In [18]:
search_results

{'kind': 'youtube#searchListResponse',
 'etag': 'm-uy_oVeYQqjWFdjK9xJby2wOmA',
 'nextPageToken': 'CAoQAA',
 'regionCode': 'US',
 'pageInfo': {'totalResults': 54767, 'resultsPerPage': 10},
 'items': [{'kind': 'youtube#searchResult',
   'etag': 'YrJloXYds7DQIHPKnCY8fRn44oU',
   'id': {'kind': 'youtube#playlist',
    'playlistId': 'PLsyeobzWxl7poL9JTVyndKe62ieoN-MZ3'},
   'snippet': {'publishedAt': '2018-06-30T13:24:43Z',
    'channelId': 'UC59K-uG2A5ogwIrHw4bmlEg',
    'title': 'Python for Beginners (Full Course) | Programming Tutorial',
    'description': 'Python Tutorial, Easy Python tutorial for beginner, learn Python Programming, learn python programming with example and ...',
    'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/QXeEoD0pB3E/default.jpg',
      'width': 120,
      'height': 90},
     'medium': {'url': 'https://i.ytimg.com/vi/QXeEoD0pB3E/mqdefault.jpg',
      'width': 320,
      'height': 180},
     'high': {'url': 'https://i.ytimg.com/vi/QXeEoD0pB3E/hqdefault

In [27]:
channel_search = ChannelSearch('python flask')
search_results = channel_search.search_channel(youtube_client)

In [28]:
search_results

{'kind': 'youtube#searchListResponse',
 'etag': 'ACiqqrx__vVq6QYEA88gjmZZAN0',
 'nextPageToken': 'CAoQAA',
 'regionCode': 'US',
 'pageInfo': {'totalResults': 11137, 'resultsPerPage': 10},
 'items': [{'kind': 'youtube#searchResult',
   'etag': 'tDD_wA-XDDWhZJxYDNJ6caL4Iog',
   'id': {'kind': 'youtube#channel', 'channelId': 'UC-QDfvrRIDB6F0bIO4I4HkQ'},
   'snippet': {'publishedAt': '2015-11-18T21:40:58Z',
    'channelId': 'UC-QDfvrRIDB6F0bIO4I4HkQ',
    'title': 'Pretty Printed',
    'description': "I'm Anthony. I make programming videos to hopefully make some things easier to understand. Need one-on-one help with your ...",
    'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/ytc/AL5GRJUeiFQvKAdPjEBHubIQttTd8JiL8IWpv8l0n-yS=s88-c-k-c0xffffffff-no-rj-mo'},
     'medium': {'url': 'https://yt3.ggpht.com/ytc/AL5GRJUeiFQvKAdPjEBHubIQttTd8JiL8IWpv8l0n-yS=s240-c-k-c0xffffffff-no-rj-mo'},
     'high': {'url': 'https://yt3.ggpht.com/ytc/AL5GRJUeiFQvKAdPjEBHubIQttTd8JiL8IWpv8l0n-yS=s800-c

In [ ]:
"""
{
  "kind": "youtube#video",
  "etag": etag,
  "id": string,
  "snippet": {
    "publishedAt": datetime,
    "channelId": string,
    "title": string,
    "description": string,
    "thumbnails": {
      (key): {
        "url": string,
        "width": unsigned integer,
        "height": unsigned integer
      }
    },
    "channelTitle": string,
    "tags": [
      string
    ],
    "categoryId": string,
    "liveBroadcastContent": string,
    "defaultLanguage": string,
    "localized": {
      "title": string,
      "description": string
    },
    "defaultAudioLanguage": string
  },
  "contentDetails": {
    "duration": string,
    "dimension": string,
    "definition": string,
    "caption": string,
    "licensedContent": boolean,
    "regionRestriction": {
      "allowed": [
        string
      ],
      "blocked": [
        string
      ]
    },
    "contentRating": {
      "acbRating": string,
      "agcomRating": string,
      "anatelRating": string,
      "bbfcRating": string,
      "bfvcRating": string,
      "bmukkRating": string,
      "catvRating": string,
      "catvfrRating": string,
      "cbfcRating": string,
      "cccRating": string,
      "cceRating": string,
      "chfilmRating": string,
      "chvrsRating": string,
      "cicfRating": string,
      "cnaRating": string,
      "cncRating": string,
      "csaRating": string,
      "cscfRating": string,
      "czfilmRating": string,
      "djctqRating": string,
      "djctqRatingReasons": [,
        string
      ],
      "ecbmctRating": string,
      "eefilmRating": string,
      "egfilmRating": string,
      "eirinRating": string,
      "fcbmRating": string,
      "fcoRating": string,
      "fmocRating": string,
      "fpbRating": string,
      "fpbRatingReasons": [,
        string
      ],
      "fskRating": string,
      "grfilmRating": string,
      "icaaRating": string,
      "ifcoRating": string,
      "ilfilmRating": string,
      "incaaRating": string,
      "kfcbRating": string,
      "kijkwijzerRating": string,
      "kmrbRating": string,
      "lsfRating": string,
      "mccaaRating": string,
      "mccypRating": string,
      "mcstRating": string,
      "mdaRating": string,
      "medietilsynetRating": string,
      "mekuRating": string,
      "mibacRating": string,
      "mocRating": string,
      "moctwRating": string,
      "mpaaRating": string,
      "mpaatRating": string,
      "mtrcbRating": string,
      "nbcRating": string,
      "nbcplRating": string,
      "nfrcRating": string,
      "nfvcbRating": string,
      "nkclvRating": string,
      "oflcRating": string,
      "pefilmRating": string,
      "rcnofRating": string,
      "resorteviolenciaRating": string,
      "rtcRating": string,
      "rteRating": string,
      "russiaRating": string,
      "skfilmRating": string,
      "smaisRating": string,
      "smsaRating": string,
      "tvpgRating": string,
      "ytRating": string
    },
    "projection": string,
    "hasCustomThumbnail": boolean
  },
  "status": {
    "uploadStatus": string,
    "failureReason": string,
    "rejectionReason": string,
    "privacyStatus": string,
    "publishAt": datetime,
    "license": string,
    "embeddable": boolean,
    "publicStatsViewable": boolean,
    "madeForKids": boolean,
    "selfDeclaredMadeForKids": boolean
  },
  "statistics": {
    "viewCount": string,
    "likeCount": string,
    "dislikeCount": string,
    "favoriteCount": string,
    "commentCount": string
  },
  "player": {
    "embedHtml": string,
    "embedHeight": long,
    "embedWidth": long
  },
  "topicDetails": {
    "topicIds": [
      string
    ],
    "relevantTopicIds": [
      string
    ],
    "topicCategories": [
      string
    ]
  },
  "recordingDetails": {
    "recordingDate": datetime
  },
  "fileDetails": {
    "fileName": string,
    "fileSize": unsigned long,
    "fileType": string,
    "container": string,
    "videoStreams": [
      {
        "widthPixels": unsigned integer,
        "heightPixels": unsigned integer,
        "frameRateFps": double,
        "aspectRatio": double,
        "codec": string,
        "bitrateBps": unsigned long,
        "rotation": string,
        "vendor": string
      }
    ],
    "audioStreams": [
      {
        "channelCount": unsigned integer,
        "codec": string,
        "bitrateBps": unsigned long,
        "vendor": string
      }
    ],
    "durationMs": unsigned long,
    "bitrateBps": unsigned long,
    "creationTime": string
  },
  "processingDetails": {
    "processingStatus": string,
    "processingProgress": {
      "partsTotal": unsigned long,
      "partsProcessed": unsigned long,
      "timeLeftMs": unsigned long
    },
    "processingFailureReason": string,
    "fileDetailsAvailability": string,
    "processingIssuesAvailability": string,
    "tagSuggestionsAvailability": string,
    "editorSuggestionsAvailability": string,
    "thumbnailsAvailability": string
  },
  "suggestions": {
    "processingErrors": [
      string
    ],
    "processingWarnings": [
      string
    ],
    "processingHints": [
      string
    ],
    "tagSuggestions": [
      {
        "tag": string,
        "categoryRestricts": [
          string
        ]
      }
    ],
    "editorSuggestions": [
      string
    ]
  },
  "liveStreamingDetails": {
    "actualStartTime": datetime,
    "actualEndTime": datetime,
    "scheduledStartTime": datetime,
    "scheduledEndTime": datetime,
    "concurrentViewers": unsigned long,
    "activeLiveChatId": string
  },
  "localizations": {
    (key): {
      "title": string,
      "description": string
    }
  }
}
"""

{
  "kind": "youtube#video",
  "etag": etag,
  "id": string,
  "snippet": {
    "publishedAt": datetime,
    "channelId": string,
    "title": string,
    "description": string,
    "thumbnails": {
      (key): {
        "url": string,
        "width": unsigned integer,
        "height": unsigned integer
      }
    },
    "channelTitle": string,
    "tags": [
      string
    ],
    "categoryId": string,
    "liveBroadcastContent": string,
    "defaultLanguage": string,
    "localized": {
      "title": string,
      "description": string
    },
    "defaultAudioLanguage": string
  },
  "contentDetails": {
    "duration": string,
    "dimension": string,
    "definition": string,
    "caption": string,
    "licensedContent": boolean,
    "regionRestriction": {
      "allowed": [
        string
      ],
      "blocked": [
        string
      ]
    },
    "contentRating": {
      "acbRating": string,
      "agcomRating": string,
      "anatelRating": string,
      "bbfcRating": string,
      "bfvcRating": string,
      "bmukkRating": string,
      "catvRating": string,
      "catvfrRating": string,
      "cbfcRating": string,
      "cccRating": string,
      "cceRating": string,
      "chfilmRating": string,
      "chvrsRating": string,
      "cicfRating": string,
      "cnaRating": string,
      "cncRating": string,
      "csaRating": string,
      "cscfRating": string,
      "czfilmRating": string,
      "djctqRating": string,
      "djctqRatingReasons": [,
        string
      ],
      "ecbmctRating": string,
      "eefilmRating": string,
      "egfilmRating": string,
      "eirinRating": string,
      "fcbmRating": string,
      "fcoRating": string,
      "fmocRating": string,
      "fpbRating": string,
      "fpbRatingReasons": [,
        string
      ],
      "fskRating": string,
      "grfilmRating": string,
      "icaaRating": string,
      "ifcoRating": string,
      "ilfilmRating": string,
      "incaaRating": string,
      "kfcbRating": string,
      "kijkwijzerRating": string,
      "kmrbRating": string,
      "lsfRating": string,
      "mccaaRating": string,
      "mccypRating": string,
      "mcstRating": string,
      "mdaRating": string,
      "medietilsynetRating": string,
      "mekuRating": string,
      "mibacRating": string,
      "mocRating": string,
      "moctwRating": string,
      "mpaaRating": string,
      "mpaatRating": string,
      "mtrcbRating": string,
      "nbcRating": string,
      "nbcplRating": string,
      "nfrcRating": string,
      "nfvcbRating": string,
      "nkclvRating": string,
      "oflcRating": string,
      "pefilmRating": string,
      "rcnofRating": string,
      "resorteviolenciaRating": string,
      "rtcRating": string,
      "rteRating": string,
      "russiaRating": string,
      "skfilmRating": string,
      "smaisRating": string,
      "smsaRating": string,
      "tvpgRating": string,
      "ytRating": string
    },
    "projection": string,
    "hasCustomThumbnail": boolean
  },
  "status": {
    "uploadStatus": string,
    "failureReason": string,
    "rejectionReason": string,
    "privacyStatus": string,
    "publishAt": datetime,
    "license": string,
    "embeddable": boolean,
    "publicStatsViewable": boolean,
    "madeForKids": boolean,
    "selfDeclaredMadeForKids": boolean
  },
  "statistics": {
    "viewCount": string,
    "likeCount": string,
    "dislikeCount": string,
    "favoriteCount": string,
    "commentCount": string
  },
  "player": {
    "embedHtml": string,
    "embedHeight": long,
    "embedWidth": long
  },
  "topicDetails": {
    "topicIds": [
      string
    ],
    "relevantTopicIds": [
      string
    ],
    "topicCategories": [
      string
    ]
  },
  "recordingDetails": {
    "recordingDate": datetime
  },
  "fileDetails": {
    "fileName": string,
    "fileSize": unsigned long,
    "fileType": string,
    "container": string,
    "videoStreams": [
      {
        "widthPixels": unsigned integer,
        "heightPixels": unsigned integer,
        "frameRateFps": double,
        "aspectRatio": double,
        "codec": string,
        "bitrateBps": unsigned long,
        "rotation": string,
        "vendor": string
      }
    ],
    "audioStreams": [
      {
        "channelCount": unsigned integer,
        "codec": string,
        "bitrateBps": unsigned long,
        "vendor": string
      }
    ],
    "durationMs": unsigned long,
    "bitrateBps": unsigned long,
    "creationTime": string
  },
  "processingDetails": {
    "processingStatus": string,
    "processingProgress": {
      "partsTotal": unsigned long,
      "partsProcessed": unsigned long,
      "timeLeftMs": unsigned long
    },
    "processingFailureReason": string,
    "fileDetailsAvailability": string,
    "processingIssuesAvailability": string,
    "tagSuggestionsAvailability": string,
    "editorSuggestionsAvailability": string,
    "thumbnailsAvailability": string
  },
  "suggestions": {
    "processingErrors": [
      string
    ],
    "processingWarnings": [
      string
    ],
    "processingHints": [
      string
    ],
    "tagSuggestions": [
      {
        "tag": string,
        "categoryRestricts": [
          string
        ]
      }
    ],
    "editorSuggestions": [
      string
    ]
  },
  "liveStreamingDetails": {
    "actualStartTime": datetime,
    "actualEndTime": datetime,
    "scheduledStartTime": datetime,
    "scheduledEndTime": datetime,
    "concurrentViewers": unsigned long,
    "activeLiveChatId": string
  },
  "localizations": {
    (key): {
      "title": string,
      "description": string
    }
  }
}

In [ ]:
"""
The keys include:
1. kind
2. etag
3. id
4. snippet
5. contentDetails
6. status
7. statistics
8. player
9. topicDetails
10. recordingDetails
11. fileDetails
12. processingDetails
13. suggestions
14. liveStreamingDetails
15. localizations

snippet, contentDetails, statistics
"""

In [14]:
search_request = youtube_client.videos().list(
    id='rfscVS0vtbw',
    part='snippet,contentDetails,statistics'
)
search_response = search_request.execute()

In [15]:
search_response

{'kind': 'youtube#videoListResponse',
 'etag': '57TLDMP6XyyCKM9ZC7M6C6HT6lM',
 'items': [{'kind': 'youtube#video',
   'etag': 'gekfGVBsUWEBKyUS9s5yOdKtDXw',
   'id': 'rfscVS0vtbw',
   'snippet': {'publishedAt': '2018-07-11T18:00:42Z',
    'channelId': 'UC8butISFwT-Wl7EV0hUK0BQ',
    'title': 'Learn Python - Full Course for Beginners [Tutorial]',
    'description': "This course will give you a full introduction into all of the core concepts in python. Follow along with the videos and you'll be a python programmer in no time!\nWant more from Mike? He's starting a coding RPG/Bootcamp - https://simulator.dev/\n\n⭐️ Contents ⭐\n⌨️ (0:00) Introduction\n⌨️ (1:45) Installing Python & PyCharm\n⌨️ (6:40) Setup & Hello World\n⌨️ (10:23) Drawing a Shape\n⌨️ (15:06) Variables & Data Types\n⌨️ (27:03) Working With Strings\n⌨️ (38:18) Working With Numbers\n⌨️ (48:26) Getting Input From Users\n⌨️ (52:37) Building a Basic Calculator\n⌨️ (58:27) Mad Libs Game\n⌨️ (1:03:10) Lists\n⌨️ (1:10:44) List Funct

In [ ]:
class YouTubeVideo:
    

In [149]:
class YouTubeVideoStats:
    def __init__(self, video_id: str):
        self.__video_id = video_id
        
    def get_stats():
        pass
    
    def __generate_basic_stats_params(self):
        basic_stats_params = dict(
            part="statistics,contentDetails",
            id=self.__video_id,
            fields="items(statistics,contentDetails(duration))"
        )
        return basic_stats_params
    
    def search_video(self, youtube_client, stats_type='basic'):
        search_response = None
        if stats_type == 'basic':
            basic_stats_params = self.__generate_basic_stats_params()
            search_request = youtube_client.videos().list(
                **basic_stats_params
            )
            search_response = search_request.execute()
        return search_response
    
    @staticmethod
    def parse_basic_response(search_result: dict) -> dict[str, str]:
        video_stats = {}
        return video_stats

In [150]:
video_stats = YouTubeVideoStats('rfscVS0vtbw')
video_stats.search_video(youtube_client)

{'items': [{'contentDetails': {'duration': 'PT4H26M52S'},
   'statistics': {'viewCount': '39007381',
    'likeCount': '946011',
    'favoriteCount': '0',
    'commentCount': '43172'}}]}

{
  "kind": "youtube#playlist",
  "etag": etag,
  "id": string,
  "snippet": {
    "publishedAt": datetime,
    "channelId": string,
    "title": string,
    "description": string,
    "thumbnails": {
      (key): {
        "url": string,
        "width": unsigned integer,
        "height": unsigned integer
      }
    },
    "channelTitle": string,
    "defaultLanguage": string,
    "localized": {
      "title": string,
      "description": string
    }
  },
  "status": {
    "privacyStatus": string
  },
  "contentDetails": {
    "itemCount": unsigned integer
  },
  "player": {
    "embedHtml": string
  },
  "localizations": {
    (key): {
      "title": string,
      "description": string
    }
  }
}

In [ ]:
"""
{
  "kind": "youtube#playlist",
  "etag": etag,
  "id": string,
  "snippet": {
    "publishedAt": datetime,
    "channelId": string,
    "title": string,
    "description": string,
    "thumbnails": {
      (key): {
        "url": string,
        "width": unsigned integer,
        "height": unsigned integer
      }
    },
    "channelTitle": string,
    "defaultLanguage": string,
    "localized": {
      "title": string,
      "description": string
    }
  },
  "status": {
    "privacyStatus": string
  },
  "contentDetails": {
    "itemCount": unsigned integer
  },
  "player": {
    "embedHtml": string
  },
  "localizations": {
    (key): {
      "title": string,
      "description": string
    }
  }
}
"""

In [21]:
search_request = youtube_client.playlists().list(
    id='PLsyeobzWxl7poL9JTVyndKe62ieoN-MZ3',
    part='snippet,contentDetails'
)
search_response = search_request.execute()

In [22]:
search_response

{'kind': 'youtube#playlistListResponse',
 'etag': 'lZrI6pCAeeFxYTCqM9uq70xUyjQ',
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#playlist',
   'etag': 'PHuGZluJrMOCfhvWHvk1XX7h18I',
   'id': 'PLsyeobzWxl7poL9JTVyndKe62ieoN-MZ3',
   'snippet': {'publishedAt': '2018-06-30T13:24:43Z',
    'channelId': 'UC59K-uG2A5ogwIrHw4bmlEg',
    'title': 'Python for Beginners (Full Course) | Programming Tutorial',
    'description': 'Python Tutorial, Easy Python tutorial for beginner, learn Python Programming, learn python programming with example and syntax. Download python, install python, using python IDE, IDLE, Pycharm IDE, notepad,  getting started with python, variables, functions, object, list in python, set in python, help in python, data types, operators, math function, run, debug python code, conditional statement, array, matrix in python, python by navin reddy',
    'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/QXeEoD0pB3E/default.jpg',
      

In [35]:
search_request = youtube_client.channels().list(
    id='UC-QDfvrRIDB6F0bIO4I4HkQ',
    part='snippet,contentDetails,statistics,topicDetails'
)
search_response = search_request.execute()

In [36]:
search_response

{'kind': 'youtube#channelListResponse',
 'etag': 'd8hbbLRrt-QheRiBOrBBPqzUvpk',
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#channel',
   'etag': 'GH-klOW_Hk1zNLnhEdsQmKpp_yU',
   'id': 'UC-QDfvrRIDB6F0bIO4I4HkQ',
   'snippet': {'title': 'Pretty Printed',
    'description': "I'm Anthony. I make programming videos to hopefully make some things easier to understand.\n\nNeed one-on-one help with your project? I can help through my coaching program. Learn more here: https://prettyprinted.com/coaching",
    'customUrl': '@prettyprinted',
    'publishedAt': '2015-11-18T21:40:58Z',
    'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/ytc/AL5GRJUeiFQvKAdPjEBHubIQttTd8JiL8IWpv8l0n-yS=s88-c-k-c0x00ffffff-no-rj',
      'width': 88,
      'height': 88},
     'medium': {'url': 'https://yt3.ggpht.com/ytc/AL5GRJUeiFQvKAdPjEBHubIQttTd8JiL8IWpv8l0n-yS=s240-c-k-c0x00ffffff-no-rj',
      'width': 240,
      'height': 240},
     'high': {'url': 'https://yt3.g

In [38]:
spider_man = search_video()

In [39]:
spider_man

{'kind': 'youtube#searchListResponse',
 'etag': 'e5_ZI3NMxWDspDvbuuORnXZXTus',
 'nextPageToken': 'CAoQAA',
 'regionCode': 'KE',
 'pageInfo': {'totalResults': 1000000, 'resultsPerPage': 10},
 'items': [{'kind': 'youtube#searchResult',
   'etag': 'nnwDqUXMNsScosopUGk7IYC3G6c',
   'id': {'kind': 'youtube#video', 'videoId': 'uKYV2qjYIS0'},
   'snippet': {'publishedAt': '2023-02-20T17:43:03Z',
    'channelId': 'UC-pnSdXRsSi44apSO5RXjlw',
    'title': 'Spooder-Man: Across The Spooder-Verse Trailer',
    'description': 'SPOODERMAN MERCH : https://shop.laughoverlife.com/ In this newest official unofficial movie trailer featuring our special ...',
    'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/uKYV2qjYIS0/default.jpg',
      'width': 120,
      'height': 90},
     'medium': {'url': 'https://i.ytimg.com/vi/uKYV2qjYIS0/mqdefault.jpg',
      'width': 320,
      'height': 180},
     'high': {'url': 'https://i.ytimg.com/vi/uKYV2qjYIS0/hqdefault.jpg',
      'width': 480,
      'height'

In [21]:
url = 'https://www.youtube.com/watch?v=Dqdu-FsBk0s'
youtube.find_video(url)

url


'Dqdu-FsBk0s'

In [ ]:
"""
I want to be able to fetch:
1. Get a single youtube video
2. Get a playlist
3. Get a single channel
4. Get comments
5. Get video stats
6. Get channel stats
"""

In [ ]:
"""
The items that can be fetched frm the YouTube API arecalled Resources.
The Resources include:
1. Activities
2. Captions
3. Channels
4. ChannelSectons
5. Comments
6. CommentThreads

Each Resource supports various methods of most importance is the list method. They also
have a representation, which is the response returned by the API. An example video 
representation is:


"""

In [ ]:
"""
Resources: 
---------
YouTube's entities from which you can extract information.

Methods: 
-------
Functions which can be applied to the resources in order to extract their properties 
or modify them.

List methods: 
------------
This is the method of our interest since it can be used to retrieve collections of 
resources and then extract information about each one of them.

Overview section: 
----------------
Where the resource is explained. The most interesting part here is the Resource 
representation subsection. It will be useful since from there you will select the 
particular properties you want to retrieve from the resources you request.

list section: 
------------
Where the list method for each resource is explained.
— In the Parameters subsection you can see the required and optional parameters that 
can be used in order to customize the queries.
— In the Response subsection you can see the Response representation which includes 
the properties the query response has. It is important because you can also retrieve 
or ignore attributes of the responses.
"""

In [ ]:
"""
Video Resource Response
-----------------------

{
  "kind": "youtube#videoListResponse",
  "etag": etag,
  "nextPageToken": string,
  "prevPageToken": string,
  "pageInfo": {
    "totalResults": integer,
    "resultsPerPage": integer
  },
  "items": [
    video Resource
  ]
}

Errors
------
badRequest (400) videoChartNotFound 
The requested video chart is not supported or is not available.

forbidden (403) forbidden
The request is not properly authorized to access video file or processing information. 
Note that the fileDetails, processingDetails, and suggestions parts are only available 
to that video's owner.

forbidden (403) forbidden
The request cannot access user rating information. This error may occur because the 
request is not properly authorized to use the myRating parameter.

notFound (404) videoNotFound
The video that you are trying to retrieve cannot be found. Check the value of the 
request's id parameter to ensure that it is correct.

"""

In [3]:
query = Query("Python")
query.query_string

'Python'

In [ ]:
class YouTubeVideo:
    """A YouTube Video."""
    def __init__(self):
        self.__video_id
        self.__stats = YouTubeVideoStats(self.__video_id)
        self.__details = YouTubeDetails(self.__video_id)
        self.__comments = YouTubeComments(self.__video_id)
        
    def get_stats(self):
        pass
    
    def get_details(self):
        pass
    
    def get_comments(self):
        pass

In [ ]:
class YouTubeChannel:
    pass

In [ ]:
class YouTubePlayList

In [ ]:
class YouTubeCommentThread:
    pass

In [ ]:
class Stats:
    pass

class YouTubeVideoStats:
    pass

class YouTubeChannelStats:
    pass

In [ ]:
class YouTubeComment:
    pass

In [38]:
def get_video_details(youtube, **kwargs):
    return youtube.videos().list(
        part="snippet,contentDetails,statistics",
        **kwargs
    ).execute()

In [39]:
def print_video_infos(video_response):
    items = video_response.get("items")[0]
    # get the snippet, statistics & content details from the video response
    snippet         = items["snippet"]
    statistics      = items["statistics"]
    content_details = items["contentDetails"]
    # get infos from the snippet
    channel_title = snippet["channelTitle"]
    title         = snippet["title"]
    description   = snippet["description"]
    publish_time  = snippet["publishedAt"]
    # get stats infos
    comment_count = statistics["commentCount"]
    like_count    = statistics["likeCount"]
    view_count    = statistics["viewCount"]
    # get duration from content details
    duration = content_details["duration"]
    # duration in the form of something like 'PT5H50M15S'
    # parsing it to be something like '5:50:15'
    parsed_duration = re.search(f"PT(\d+H)?(\d+M)?(\d+S)", duration).groups()
    duration_str = ""
    for d in parsed_duration:
        if d:
            duration_str += f"{d[:-1]}:"
    duration_str = duration_str.strip(":")
    print(f"""\
    Title: {title}
    Description: {description}
    Channel Title: {channel_title}
    Publish time: {publish_time}
    Duration: {duration_str}
    Number of comments: {comment_count}
    Number of likes: {like_count}
    Number of views: {view_count}
    """)

In [40]:
def get_video_id_by_url(url):
    """
    Return the Video ID from the video `url`
    """
    # split URL parts
    parsed_url = p.urlparse(url)
    # get the video ID by parsing the query of the URL
    video_id = p.parse_qs(parsed_url.query).get("v")
    if video_id:
        return video_id[0]
    else:
        raise Exception(f"Wasn't able to parse video URL: {url}")

In [41]:
video_url = "https://www.youtube.com/watch?v=jNQXAC9IVRw&ab_channel=jawed"
# parse video ID from URL
video_id = get_video_id_by_url(video_url)
# make API call to get video info
response = get_video_details(youtube, id=video_id)
# print extracted video infos
print_video_infos(response)

    Title: Me at the zoo
    Description: 
    Channel Title: jawed
    Publish time: 2005-04-24T03:31:52Z
    Duration: 19
    Number of comments: 11269824
    Number of likes: 13253111
    Number of views: 257732508
    
